In [1]:
# !pip freeze > requirements.txt
# !pip install -r requirements.txt

In [2]:
import os
from datetime import datetime
import glob
import json
import gc

from tqdm import tqdm
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from PIL import Image

import torch
from torch import nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
import torchvision.models as models

In [3]:
class ImageDataset(Dataset):
    def __init__(self, path, transform, crop=False):
        self.image_files = []
        self.box_locations = [] if crop else None
        self.labels = []
        self.transform = transform
        self.crop = crop

        for root, dirs, files in os.walk(path): 
            for dir in dirs:
                dir_path = os.path.join(root, dir)
                files = glob.glob(os.path.join(dir_path, '*'))
                files.sort()  

                for file in files:
                    if file.endswith('.jpg'):
                        json_file = file.replace('.jpg', '.json')

                        if json_file in files:
                            try:
                                with open(json_file, 'r') as f:
                                    json_content = json.load(f)
                                    label = 0 if json_content['metaData']['lesions'] == 'A7' else 1
                                    box = []
                                    for info in json_content['labelingInfo']:
                                        if 'box' in info:
                                            box.extend(info['box']['location'])

                                    self.image_files.append(file)
                                    self.labels.append(label)
                                    
                                    if self.crop:
                                        box = json_content['label']['label_bbox']
                                        self.box_locations.append(box)
                            except Exception as e:
                                print(f'Error processing file {json_file}: {e}')

    def calculate_crop_size(self, box_locations):
        x_coords = [box['x'] for box in box_locations]
        y_coords = [box['y'] for box in box_locations]
        widths = [box['width'] for box in box_locations]
        heights = [box['height'] for box in box_locations]

        min_x = min(x_coords)
        min_y = min(y_coords)
        max_x = max(x + width for x, width in zip(x_coords, widths))
        max_y = max(y + height for y, height in zip(y_coords, heights))

        return (min_x, min_y, max_x, max_y)

    def crop_image(self, image_path, box_locations):
        crop_size = self.calculate_crop_size(box_locations)
        image = Image.open(image_path).crop(crop_size)

        return image

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        label = self.labels[idx]

        if self.crop:
            box_locations = self.box_locations[idx]
            image = self.crop_image(image_path, box_locations) if box_locations else Image.open(image_path)
        else:
            image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)

        return image, label

class DataLoaderMaker:
    def __init__(self,
                 dataset,
                 train_ratio,
                 batch_size,
                 num_workers):
        self.dataset = dataset
        self.train_ratio = train_ratio
        self.batch_size = batch_size
        self.num_workers = num_workers

        print(f"Total dataset size: {len(self.dataset)}")
        print(f"Total positive size: {dataset.labels.count(0)}")
        print(f"Total negative size: {dataset.labels.count(1)}")

        if 1 > train_ratio > 0:
            self.split_and_make_dataloader(train_ratio)
            self.inspect_data(self.train_loader)
            self.inspect_data(self.test_loader)
        else:
            self.make_dataloader()
            self.inspect_data(self.dataloader)

    def split_and_make_dataloader(self, train_ratio):
        train_size = int(len(self.dataset) * train_ratio)
        test_size = len(self.dataset) - train_size
        self.train_dataset, self.test_dataset = random_split(self.dataset, [train_size, test_size])

        self.train_loader = DataLoader(dataset=self.train_dataset,
                                       batch_size=self.batch_size,
                                       shuffle=True,
                                       num_workers=self.num_workers)
        self.test_loader = DataLoader(dataset=self.test_dataset,
                                      batch_size=self.batch_size,
                                      shuffle=False,
                                      num_workers=self.num_workers)

    def make_dataloader(self):
        self.dataloader = DataLoader(dataset=self.dataset,
                                     batch_size=self.batch_size,
                                     shuffle=False,
                                     num_workers=self.num_workers)
        
    def inspect_data(self, dataloader):
        print("Inspecting Data...")
        images, labels = next(iter(dataloader))
        print(f"- Images shape: {images.shape}, Labels: {labels}")

In [4]:
class ModelTrainer:
    def __init__(self, 
                 model,
                 device,
                 train_dataloader,
                 valid_dataloader,
                 loss_fn,
                 optimizer,
                 scheduler):
        self.device = device
        self.model = model.to(self.device)
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.best_loss = float("inf")
        self.start_time = datetime.now().strftime('%Y%m%d-%H%M%S')
        self.writer = SummaryWriter(log_dir=f'runs/{self.start_time}')

    def train_one_epoch(self, epoch, num_epochs, accumulation_steps):
        self.model.train()
        total_loss = 0.0
        correct = 0
        total = 0
    
        for step, (inputs, labels) in enumerate(tqdm(self.train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")):
            inputs, labels = inputs.to(self.device), labels.to(self.device)
    
            outputs = self.model(inputs)
    
            loss = self.loss_fn(outputs, labels) / accumulation_steps
            total_loss += loss.item() * accumulation_steps
    
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
            loss.backward()
            
            if (step + 1) % accumulation_steps == 0 or (step + 1) == len(self.train_dataloader):
                self.optimizer.step()
                self.optimizer.zero_grad()
                
            del inputs, labels, outputs
            gc.collect()
    
        avg_loss = total_loss / len(self.train_dataloader)
        accuracy = correct / total  # Calculate accuracy
        self.writer.add_scalar("Loss/train", avg_loss, epoch)        
        self.writer.add_scalar("Accuracy/train", accuracy, epoch)
        self.scheduler.step()

    def eval_one_epoch(self, epoch):
        self.model.eval()
        total_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in self.valid_dataloader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                outputs = self.model(inputs)

                loss = self.loss_fn(outputs, labels)
                total_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                del inputs, labels, outputs
                gc.collect()

        avg_loss = total_loss / len(self.valid_dataloader)
        accuracy = correct / total
        self.writer.add_scalar("Loss/valid", avg_loss, epoch)
        self.writer.add_scalar("Accuracy/valid", accuracy, epoch)  
        
        if avg_loss < self.best_loss:
            self.best_loss = avg_loss
            torch.save(self.model.state_dict(), "best_model.pth")

        print(f"Validation Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

    def train(self, num_epochs, accumulation_steps):
        start_time = datetime.now().strftime('%Y%m%d-%H%M%S')
        
        for epoch in range(num_epochs):
            self.train_one_epoch(epoch, num_epochs, accumulation_steps)
            self.eval_one_epoch(epoch)
            
        self.writer.close()

In [5]:
path = 'skin/Camera/Train'

transform = transforms.Compose([transforms.Resize((240, 240)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

dataset = ImageDataset(path=path, transform=transform)

train_ratio = 0.9
batch_size = 64
num_workers = os.cpu_count()

data_loader_maker = DataLoaderMaker(dataset=dataset,
                                    batch_size=batch_size,
                                    train_ratio=train_ratio,
                                    num_workers=num_workers)

train_dataloader = data_loader_maker.train_loader
valid_dataloader = data_loader_maker.test_loader

Total dataset size: 610329
Inspecting Data...
- Images shape: torch.Size([64, 3, 240, 240]), Labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
        1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1])
Inspecting Data...
- Images shape: torch.Size([64, 3, 240, 240]), Labels: tensor([0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0])


In [6]:
model = models.efficientnet_b1()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 2
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, num_classes)
loss_fn=nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

trainer = ModelTrainer(model=model,
                       device=device,
                       train_dataloader=train_dataloader,
                       valid_dataloader=valid_dataloader,
                       loss_fn=loss_fn,
                       optimizer=optimizer,
                       scheduler=scheduler)

In [ ]:
trainer.train(96, 2)

Epoch 1/96:  28%|██▊       | 2397/8583 [14:34<36:27,  2.83batch/s]

In [ ]:
class ModelTester:
    def __init__(self, path, model, device, dataloader):
        self.model = model
        self.device = device
        self.dataloader = dataloader
        self.load_weights(path)
        self.evaluate()

    def load_weights(self, path):
        weights = torch.load(path)
        self.model.load_state_dict(weights)
        self.model.to(self.device)

    def classify(self):
        self.model.eval()
        predictions = []
        labels = []

        with torch.no_grad():
            for inputs, targets in tqdm(self.dataloader):
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)
                outputs = self.model(inputs)
                _, predicted = torch.max(outputs, 1)
                predictions.extend(predicted.cpu().numpy())
                labels.extend(targets.cpu().numpy())
                
        return predictions, labels

    def evaluate(self):
        predictions, labels = self.classify()
        cm = confusion_matrix(labels, predictions)
        accuracy = accuracy_score(labels, predictions)
        precision = precision_score(labels, predictions)
        recall = recall_score(labels, predictions)
        f1 = f1_score(labels, predictions)
    
        print("Evaluation Results:")
        print(f"Confusion Matrix:\n{cm}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")

In [ ]:
path = 'skin/Camera/Validation'

transform = transforms.Compose([transforms.Resize((240, 240)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

dataset = ImageDataset(path=path, transform=transform)

train_ratio = 1
batch_size = 32
num_workers = os.cpu_count()

data_loader_maker = DataLoaderMaker(dataset=dataset,
                                    batch_size=batch_size,
                                    train_ratio=train_ratio,
                                    num_workers=num_workers)

dataloader = data_loader_maker.dataloader

In [ ]:
path = 'best_model.pth'
model = models.efficientnet_b1()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 2
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, num_classes)

ModelTester(path=path, model=model, device=device, dataloader=dataloader)

In [ ]:
class ImageClassifier:
    def __init__(self, device, image_path, image_transform, model_weights_path, model):
        self.image_path = image_path
        self.image_transform = image_transform
        self.device = device
        self.model = model
        self.load_model_weights(model_weights_path)

    def preprocess_image(self):
        image = Image.open(self.image_path)
        input_data = self.image_transform(image).unsqueeze(0)
        input_data = input_data.to(self.device)
        return input_data

    def load_model_weights(self, weights_path):
        weights = torch.load(weights_path)
        self.model.load_state_dict(weights)
        self.model.to(self.device)

    def classify_image(self):
        self.model.eval()
        input_data = self.preprocess_image()

        with torch.no_grad():
            output = self.model(input_data)
        
        probabilities = torch.nn.functional.softmax(output, dim=1)
        predicted_prob, predicted_class = torch.max(probabilities, 1)
        
        class_label = "Disease" if predicted_class.item() == 0 else "Normal"
        class_prob = probabilities[0, predicted_class].item() * 100

        print(f"The image is predicted to be {class_label} with {class_prob:.2f}% probability.")
                
        return predicted_class.item(), class_prob

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_path = 'kIHjZsKQ6HdQ.jpg'
image_transform = transforms.Compose([transforms.Resize((240, 240)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

model_weights_path = 'best_model.pth'
model = models.efficientnet_b1()
num_classes = 2
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, num_classes)

classifier = ImageClassifier(device=device,
                             image_path=image_path,
                             image_transform=image_transform,
                             model_weights_path=model_weights_path,
                             model=model)

predicted_class = classifier.classify_image()